# Atividade 3: Classificação de Texto
Aluno: Rennê Ruan Alves Oliveira - Matrícula 242101209 - PPGI

Esta atividade tem como objetivo realizar a classificação de textos presente no corpo de dados. Cada corpo de texto possui uma classe/label associada previamente, ou seja serão utilizados para classificação modelos de aprendizado supervisionado.

Foi criado o arquivo `find_best_hyperparameters.py` contendo a classe `HyperparameterOptimizer`, esta ao ser declarada recebendo os dados desejados, irá realizar os seguintes passos:
- Pré-processamento de texto, retira caracteres especiais mantendo apenas letras e números, transforma todo o texto em caixa baixa e realiza a lematização do texto.
- Divide os dados após o pré-processamento em treino e teste, ressaltando que as etapas anteriores não geram *data leak* uma vez que as ações não consideram todo o escopo dos dados.
- Para cada modelo declara um objeto contendo a lista de hiperparâmetros a serem analisados, que seram combinados com o objeto com hiperparâmetros do vetorizador.
- Será criado um pipeline que irá processar os dados de teste em um vetorizador, o utilizado foi o TF-IDF, *Term Frequency-Inverse Document Frequency* que verifica a importância termo de texto em um documento.
- O pipeline contendo o vetorizador e o modelo será executado utilizando o GridSearchCV, o qual procura os melhores hiperparâmetros dentre os fornecidos para iteração.
- A classe permite acessar os dados de testes separados e processados, além dos melhores parâmetros encontrados para os modelo. É possível também exportar os resultados em um arquivo CSV. 

Para este projeto foi utilizado um ambiente de execução Python 3.11 criado a partir do gerenciador Anaconda.


In [100]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import accuracy_score, classification_report, f1_score

# Carregamos o arquivo com a classe Hyperparameter Optmizer
from find_best_hyperparameters import HyperparameterOptimizer

### Dados do Setor Industrial ("Industry Sector.csv")

In [2]:
df_industry = pd.read_csv("data/Industry Sector.csv")
df_industry.head()

,file_name,text,class
0,materials_http:^^www.littonacdipe.com,Litton Litton: Interconnect Prod...,basic
1,materials_http:^^www.weirton.com^contact.htm,"Allow: GET, HEAD URI: Contact Weirton St...",basic
2,materials_http:^^home.earthlink.net^~nuco2^Bul...,Systems Which system would you choose?...,basic
3,materials_http:^^www.useg.com^invest.html,US Energy Corporation Investor News ...,basic
4,materials_http:^^www.terraindustries.com^about...,"Terra Industries, Inc., Company Informat...",basic


A função `get_results`, retorna o dicionário contendo os alias para os algoritmos e seus respectivos resultados. Podemos utilizá-lo para verificar os melhores resultados e treinar os modelos com eles, algo que também será feito carregando o arquivo csv após exportação em momento posterior.

In [4]:
optmizer = HyperparameterOptimizer(df_industry["text"], df_industry["class"])

results = optmizer.get_results()

Tamanho de treino: (7053,)
Tamanho de teste: (1764,)
Encontrando melhores parâmetros para Naive Bayes Multinomial

Lista de parâmetros utilizada: {'clf__alpha': [0.0001, 0.001, 0.1, 1, 10, 100, 1000], 'vect__max_df': (0.6, 0.8), 'vect__min_df': (1, 3, 5), 'vect__norm': ('l1', 'l2')}

Fitting 2 folds for each of 84 candidates, totalling 168 fits
Finalizada busca por melhores parâmetros para Naive Bayes Multinomial
Encontrando melhores parâmetros para Regressão Logística

Lista de parâmetros utilizada: {'clf__C': [0.001, 0.1, 1, 10, 100], 'clf__solver': ['lbfgs', 'liblinear', 'newton-cg'], 'vect__max_df': (0.6, 0.8), 'vect__min_df': (1, 3, 5), 'vect__norm': ('l1', 'l2')}

Fitting 2 folds for each of 180 candidates, totalling 360 fits
Finalizada busca por melhores parâmetros para Regressão Logística
Encontrando melhores parâmetros para SGDC

Lista de parâmetros utilizada: {'clf__alpha': [0.0001, 0.001, 0.01, 0.1], 'vect__max_df': (0.6, 0.8), 'vect__min_df': (1, 3, 5), 'vect__norm': ('l1',

O objeto results criado na classe `HyperparameterOptimizer` pode ser visualizado a seguir

In [5]:
results

{'multi_nb': GridSearchCV(cv=2,
              estimator=Pipeline(steps=[('vect',
                                         TfidfVectorizer(stop_words='english')),
                                        ('clf', MultinomialNB())]),
              n_jobs=-1,
              param_grid={'clf__alpha': [0.0001, 0.001, 0.1, 1, 10, 100, 1000],
                          'vect__max_df': (0.6, 0.8), 'vect__min_df': (1, 3, 5),
                          'vect__norm': ('l1', 'l2')},
              verbose=2),
 'logistic_regression': GridSearchCV(cv=2,
              estimator=Pipeline(steps=[('vect',
                                         TfidfVectorizer(stop_words='english')),
                                        ('clf', LogisticRegression())]),
              n_jobs=-1,
              param_grid={'clf__C': [0.001, 0.1, 1, 10, 100],
                          'clf__solver': ['lbfgs', 'liblinear', 'newton-cg'],
                          'vect__max_df': (0.6, 0.8), 'vect__min_df': (1, 3, 5),
           

Além de podermos resgatar os melhores hiperparâmetros por meio de results, que contém os objetos GridSearchCV para cada modelo, podemos exportá-los para um arquivo CSV, o qual pode ser enviado e reutilizado em outras aplicações. Para exemplificar iremos abrir o CSV apenas para o algoritmo Naive Bayes.

In [65]:
optmizer.export_grid_results(path="grid_results_industry")

In [66]:
multi_nb_results_csv = pd.read_csv(
    "grid_results_industry/multi_nb_results.csv", index_col=0
)

In [67]:
sorted_multi_nb_results = multi_nb_results_csv.sort_values(by="rank_test_score")
sorted_multi_nb_results.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf__alpha,param_vect__max_df,param_vect__min_df,param_vect__norm,params,split0_test_score,split1_test_score,mean_test_score,std_test_score,rank_test_score
13,4.121522,0.282217,1.829720,0.370751,0.0010,0.6,1,l2,"{'clf__alpha': 0.001, 'vect__max_df': 0.6, 've...",0.831018,0.830970,0.830994,0.000024,1
19,2.521844,0.455082,4.399191,0.260114,0.0010,0.8,1,l2,"{'clf__alpha': 0.001, 'vect__max_df': 0.8, 've...",0.831018,0.830970,0.830994,0.000024,1
7,2.003583,0.225026,2.869977,0.757293,0.0001,0.8,1,l2,"{'clf__alpha': 0.0001, 'vect__max_df': 0.8, 'v...",0.805500,0.813386,0.809443,0.003943,3
1,3.457021,0.074483,2.636885,0.149793,0.0001,0.6,1,l2,"{'clf__alpha': 0.0001, 'vect__max_df': 0.6, 'v...",0.805500,0.813386,0.809443,0.003943,3
21,2.447621,0.274956,3.148717,0.314029,0.0010,0.8,3,l2,"{'clf__alpha': 0.001, 'vect__max_df': 0.8, 've...",0.775163,0.781906,0.778534,0.003371,5


Podemos perceber pelo DataFrame anterior que o primeiro e segundo registro apresentam o mesmo resultado de médias, em que os treinamentos são variados apenas pelo parâmetro `vect__max_df` utilizado pelo vetorizado, mas os resultados não diferem para 0.6 ou 0.8.

In [68]:
# Sendo o melhor elemento
best_nb_result = sorted_multi_nb_results.iloc[0]
print(f'Parâmetros utilizados para melhor resultado {best_nb_result["params"]}')
print(
    f'Média dos fits para melhor resultado: {best_nb_result["mean_test_score"].round(3)}'
)

Parâmetros utilizados para melhor resultado {'clf__alpha': 0.001, 'vect__max_df': 0.6, 'vect__min_df': 1, 'vect__norm': 'l2'}
Média dos fits para melhor resultado: 0.831


Como armazenamos os resultados do GridSearch também em um dicionário em nossa classe `HyperparameterOptimizer`, vamos acessá-los diretamente para facilitar a reutilização dos melhores parâmetros. Assim como também reutilizar os dados de teste previamente separados e tratados.

In [29]:
alias = {
    "multi_nb": "Multinomial Naive Bayes",
    "logistic_regression": "Regressão Logística",
    "sgdc": "Classificador SGD",
}

for key in results:
    print(f"Resultados de score e parâmetros para o modelo {alias[key]}:")
    print(results[key].best_score_)
    print(results[key].best_params_)
    print("\n")

Resultados de score e parâmetros para o modelo Multinomial Naive Bayes:
0.8309938999060967
{'clf__alpha': 0.001, 'vect__max_df': 0.6, 'vect__min_df': 1, 'vect__norm': 'l2'}


Resultados de score e parâmetros para o modelo Regressão Logística:
0.843329056572095
{'clf__C': 100, 'clf__solver': 'liblinear', 'vect__max_df': 0.6, 'vect__min_df': 1, 'vect__norm': 'l2'}


Resultados de score e parâmetros para o modelo Classificador SGD:
0.8560897852897533
{'clf__alpha': 0.0001, 'vect__max_df': 0.6, 'vect__min_df': 1, 'vect__norm': 'l2'}




Percebemos que os resultados de parâmetros para o Vetorizador se mantém identicos entre os 3 modelos, levando em conta o que foi comentado anteriormente, que os resultados para o Naive Bayes são identicos para os valores de max_df = 0.8. Tal constância pode justificar retiramos do pipeline repassado ao GridSearchCV visando otimizar o tempo de treino.

Iremos agora aplicar os resultados aos modelos separadamente.

In [32]:
vectorizer = TfidfVectorizer(max_df=0.8, min_df=1)

X_train_vectorized = vectorizer.fit_transform(
    optmizer.X_train
)  # Resgata texto pré-processado e dividido do objeto optmizer
terms = vectorizer.get_feature_names_out()
print(f"Quantia de features vetorizadas para texto de treino: {len(terms)}")


# Para evitar data leak apenas passamos os dados de teste para o transform e não para o feat
X_test_vectorized = vectorizer.transform(optmizer.X_test)

Quantia de features vetorizadas para texto de treino: 104943


Como os melhores parâmetros retornados pelo estimador apresentam também os parâmetros do Vetorizador, vamos criar a função `remove_vect_param` para obtermos apenas os hiperparâmetros inerentes aos modelos.

In [38]:
def remove_vect_params(params_dict):
    model_params = {
        key.replace("clf__", ""): value
        for key, value in params_dict.items()
        if "clf__" in key
    }

    return model_params


print(results["multi_nb"].best_params_)
print(remove_vect_params(results["multi_nb"].best_params_))

{'clf__alpha': 0.001, 'vect__max_df': 0.6, 'vect__min_df': 1, 'vect__norm': 'l2'}
{'alpha': 0.001}


### Treinamento de Naive Bayes

In [40]:
nb_clf = MultinomialNB(**remove_vect_params(results["multi_nb"].best_params_))
# ** Se trata de um descompressor de objeto, ou seja irá transformar as chaves do dicionário em parâmetros

nb_clf.fit(X_train_vectorized, optmizer.y_train)

MultinomialNB(alpha=0.001)

In [106]:
# Função utilizada para facilitar a impressão dos scores
def print_scores(expected, pred):
    print("Acurácia:", accuracy_score(expected, pred))
    print(
        "F1-Score macro:",
        f1_score(expected, pred, average="macro"),
    )
    print(
        "F1-Score micro:",
        f1_score(expected, pred, average="micro"),
    )

In [107]:
y_pred_nb = nb_clf.predict(X_test_vectorized)
print_scores(optmizer.y_test, y_pred_nb)

Acurácia: 0.8645124716553289
F1-Score macro: 0.8634430012437485
F1-Score micro: 0.8645124716553289


### Treinamento de Regressão Logística

In [47]:
lr_clf = LogisticRegression(
    **remove_vect_params(results["logistic_regression"].best_params_)
)


lr_clf.fit(X_train_vectorized, optmizer.y_train)

LogisticRegression(C=100, solver='liblinear')

In [108]:
y_pred_lr = lr_clf.predict(X_test_vectorized)
print(y_pred_lr)

print_scores(optmizer.y_test, y_pred_lr)

['consumer' 'services' 'financial' ... 'energy' 'financial' 'basic']
Acurácia: 0.8905895691609977
F1-Score macro: 0.8937926066683711
F1-Score micro: 0.8905895691609977


### Treinamento de Classificador SGD

In [58]:
sgd_clf = SGDClassifier(**remove_vect_params(results["sgdc"].best_params_))
sgd_clf.fit(X_train_vectorized, optmizer.y_train)

SGDClassifier()

In [109]:
y_pred_sgd = sgd_clf.predict(X_test_vectorized)
print(y_pred_sgd)

print_scores(optmizer.y_test, y_pred_sgd)

['consumer' 'services' 'financial' ... 'energy' 'financial' 'basic']
Acurácia: 0.9041950113378685
F1-Score macro: 0.910484477872388
F1-Score micro: 0.9041950113378685


Temos que para os dados utilizados de Setor Industrial o modelo SGD obteve a maior acurácia (0.9). Temos que todos os modelos obtiveram, quando comparados aos resultados do GridSearch, acurácias maiores em um treinamento posterior utilizando os melhores hiperparâmetros, isso provavelmente se deve a estratégia de Cross-fold validation utilizada pelo otimizador.

### Dados de Saúde ("Dmoz-Health.csv")

Iremos replicar os mesmos passos para o dataset de dados de saúde. Para deixar a abordagem mais direto iremos utilizar direto os dados armazenados no obeto da classe otimizadora.

In [69]:
df_health = pd.read_csv("data/Dmoz-Health.csv")
df_health.head()

,file_name,text,class
0,1578510.txt,Illinois Church Action on Alcohol and Addictio...,Addictions
1,1577747.txt,AA Statewide Meeting lists for all of Vermont....,Addictions
2,1578166.txt,Gracer Medical Group Dr. Richard Gracer is a p...,Addictions
3,1577381.txt,"Phoenix Meetings, events, and visitor informat...",Addictions
4,1578793.txt,American River Area Narcotics Anonymous Resour...,Addictions


In [70]:
optmizer_health = HyperparameterOptimizer(df_health["text"], df_health["class"])

results_health = optmizer_health.get_results()

Tamanho de treino: (5200,)
Tamanho de teste: (1300,)
Encontrando melhores parâmetros para Naive Bayes Multinomial

Lista de parâmetros utilizada: {'clf__alpha': [0.0001, 0.001, 0.1, 1, 10, 100, 1000], 'vect__max_df': (0.6, 0.8), 'vect__min_df': (1, 3, 5), 'vect__norm': ('l1', 'l2')}

Fitting 2 folds for each of 84 candidates, totalling 168 fits
Finalizada busca por melhores parâmetros para Naive Bayes Multinomial
Encontrando melhores parâmetros para Regressão Logística

Lista de parâmetros utilizada: {'clf__C': [0.001, 0.1, 1, 10, 100], 'clf__solver': ['lbfgs', 'liblinear', 'newton-cg'], 'vect__max_df': (0.6, 0.8), 'vect__min_df': (1, 3, 5), 'vect__norm': ('l1', 'l2')}

Fitting 2 folds for each of 180 candidates, totalling 360 fits
Finalizada busca por melhores parâmetros para Regressão Logística
Encontrando melhores parâmetros para SGDC

Lista de parâmetros utilizada: {'clf__alpha': [0.0001, 0.001, 0.01, 0.1], 'vect__max_df': (0.6, 0.8), 'vect__min_df': (1, 3, 5), 'vect__norm': ('l1',

In [72]:
optmizer_health.export_grid_results(path="grid_results_health")

In [74]:
for key in results_health:
    print(f"Resultados de score e parâmetros para o modelo {alias[key]}:")
    print(results_health[key].best_score_)
    print(results_health[key].best_params_)
    print("\n")

Resultados de score e parâmetros para o modelo Multinomial Naive Bayes:
0.8034615384615384
{'clf__alpha': 0.1, 'vect__max_df': 0.6, 'vect__min_df': 1, 'vect__norm': 'l1'}


Resultados de score e parâmetros para o modelo Regressão Logística:
0.8275
{'clf__C': 100, 'clf__solver': 'lbfgs', 'vect__max_df': 0.6, 'vect__min_df': 1, 'vect__norm': 'l1'}


Resultados de score e parâmetros para o modelo Classificador SGD:
0.8211538461538461
{'clf__alpha': 0.0001, 'vect__max_df': 0.6, 'vect__min_df': 1, 'vect__norm': 'l2'}




In [89]:
vectorizer_health = TfidfVectorizer(max_df=0.8, min_df=1)

X_train_health_vectorized = vectorizer_health.fit_transform(optmizer_health.X_train)
terms = vectorizer_health.get_feature_names_out()
print(f"Quantia de features vetorizadas para texto de treino: {len(terms)}")

X_test_health_vectorized = vectorizer_health.transform(optmizer_health.X_test)

Quantia de features vetorizadas para texto de treino: 12141


Podemos observar como a quantia de features obtidas pelo vetorizador é um fator crucial para o tempo de treinamento do modelo.

### Treinamento de Naive Bayes

In [90]:
health_nb_clf = MultinomialNB(
    **remove_vect_params(results_health["multi_nb"].best_params_)
)

health_nb_clf.fit(X_train_health_vectorized, optmizer_health.y_train)

MultinomialNB(alpha=0.1)

In [110]:
health_y_pred_nb = health_nb_clf.predict(X_test_health_vectorized)
print(health_y_pred_nb)
print_scores(optmizer_health.y_test, health_y_pred_nb)

['Nursing' 'Senior' 'Conditions' ... 'Nutrition' 'Alternative'
 'Addictions']
Acurácia: 0.8123076923076923
F1-Score macro: 0.8130730251764948
F1-Score micro: 0.8123076923076923


### Treinamento de Regressão Logística

In [92]:
health_lr_clf = LogisticRegression(
    **remove_vect_params(results_health["logistic_regression"].best_params_)
)

health_lr_clf.fit(X_train_health_vectorized, optmizer_health.y_train)

LogisticRegression(C=100)

In [112]:
health_y_pred_lr = health_lr_clf.predict(X_test_health_vectorized)
print(health_y_pred_lr)
print_scores(optmizer_health.y_test, health_y_pred_lr)

['Nursing' 'Senior' 'Conditions' ... 'Mental' 'Alternative' 'Addictions']
Acurácia: 0.8607692307692307
F1-Score macro: 0.8592228582843957
F1-Score micro: 0.8607692307692307


### Treinamento de Classificador SGD

In [94]:
health_sgd_clf = SGDClassifier(
    **remove_vect_params(results_health["sgdc"].best_params_)
)

health_sgd_clf.fit(X_train_health_vectorized, optmizer_health.y_train)

SGDClassifier()

In [113]:
health_y_pred_sgd = health_sgd_clf.predict(X_test_health_vectorized)
print(health_y_pred_sgd)
print_scores(optmizer_health.y_test, health_y_pred_sgd)

['Nursing' 'Senior' 'Conditions' ... 'Conditions' 'Alternative'
 'Addictions']
Acurácia: 0.8646153846153846
F1-Score macro: 0.8613887820395624
F1-Score micro: 0.8646153846153846


Para os dados de saúde as acurácias foram quase idênticas para os modelos de SGD e Regressão Logística. Temos que o modelo SGD apresenta melhores resultados com sua configuração básica.


Todos os modelos apresentaram resultados de F1-score Macro e Micro muito próximos.